In [1]:
import pandas as pd
import optuna
import yfinance as yf
import pandas_datareader.data as web
from datetime import datetime
import numpy as np
import pandas_ta as ta
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.optimizers import Nadam
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

In [2]:
import warnings
warnings.filterwarnings('ignore', category = pd.errors.PerformanceWarning)

In [3]:
# Pobieramy dane dotyczące BTC z yahoo
BTC = yf.download('BTC-USD', start = '2018-01-01', end = '2019-12-31')

BTC['Target'] = (BTC['Close'].diff() > 0).astype(int)

def add_lagged_features(data, column_name, n_days):
    for i in range(1, n_days + 1):
        lagged_column_name = f'{column_name}_{i}d_back'
        data[lagged_column_name] = data[column_name].shift(i)
    return data

BTC = add_lagged_features(BTC, 'Close', 30)
BTC = add_lagged_features(BTC, 'Volume', 30)

# Usuwamy pierwszych 30 wierszy
BTC = BTC.dropna().reset_index(drop = True)

# print(BTC.head())
# print(len(BTC))
# print(BTC.shape)

[*********************100%%**********************]  1 of 1 completed


In [4]:
# Konwersacja zmiennej Target na zmienną binarną
target = to_categorical(BTC['Target'].values)

# Usuwamy niepotrzebne kolumny
features = BTC.drop(['Target', 'Adj Close'], axis = 1).values 

# Dzielimy dane na treningowe i testowe
split_idx = int(len(features) * 0.8)

m = int(np.floor(0.8 * len(features)))
train_features = features[:m]
test_features = features[m:]
train_target = target[:m]
test_target = target[m:]

In [5]:
##################################################
##################################################
# WERSJA MAŁO SKOMPLIKOWANA
##################################################
##################################################

In [6]:
# Budujemy model
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (train_features.shape[1],)),
    Dense(64, activation = 'relu'),
    Dense(2, activation = 'softmax')
])

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Trenujemy model
history = model.fit(train_features, train_target, epochs = 50, batch_size = 12, validation_split = 0.2)

C:\Users\48798\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.4841 - loss: 994302720.0000 - val_accuracy: 0.5446 - val_loss: 667419520.0000
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5454 - loss: 143705472.0000 - val_accuracy: 0.4196 - val_loss: 534271936.0000
Epoch 3/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5825 - loss: 131994824.0000 - val_accuracy: 0.4196 - val_loss: 882118272.0000
Epoch 4/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5908 - loss: 108288768.0000 - val_accuracy: 0.4464 - val_loss: 244475216.0000
Epoch 5/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5430 - loss: 79586640.0000 - val_accuracy: 0.5536 - val_loss: 272233504.0000
Epoch 6/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5746 - loss: 81269384.0000 - val_accuracy: 0.4196 - val_loss: 394203360.0000
Epoch 7/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5190 - loss: 57098336.0000 - val_accuracy: 0.4286 - val_loss: 452313920.0000
Epoch 8/

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5088 - loss: 24917564.0000 - val_accuracy: 0.4643 - val_loss: 65960228.0000
Epoch 48/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6624 - loss: 6815228.5000 - val_accuracy: 0.4554 - val_loss: 50190240.0000
Epoch 49/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5835 - loss: 11129911.0000 - val_accuracy: 0.4018 - val_loss: 81501976.0000
Epoch 50/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5898 - loss: 9339911.0000 - val_accuracy: 0.4643 - val_loss: 48453412.0000


In [7]:
# Predykcja na zbiorze treningowym
train_predictions = model.predict(train_features)
train_predictions = np.argmax(train_predictions, axis = 1)
train_true = np.argmax(train_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(train_true, train_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
138
189
67
165
Accuracy (ACC): 0.5849731663685152
True Positive Rate (TPR): 0.45544554455445546
False Positive Rate (FPR): 0.26171875
True Negative Rate (TNR): 0.73828125
Positive Predictive Value (PPV): 0.6731707317073171
Negative Predictive Value (NPV): 0.5338983050847458


In [8]:
# Predykcja na zbiorze testowym
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis = 1)
test_true = np.argmax(test_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(test_true, test_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
23
54
27
36
Accuracy (ACC): 0.55
True Positive Rate (TPR): 0.3898305084745763
False Positive Rate (FPR): 0.3333333333333333
True Negative Rate (TNR): 0.6666666666666666
Positive Predictive Value (PPV): 0.46
Negative Predictive Value (NPV): 0.6


In [9]:
##################################################
##################################################
##################################################
##################################################

In [10]:
##################################################
##################################################
# WERSJA BARDZIEJ SKOMPLIKOWANA
##################################################
##################################################

In [11]:
# Korzystamy z biblioteki OPTUNA, która służy do automatycznego dostrajania hiperparametrów
def create_model(trial):
    # Liczba warstw ukrytych, od 1 do 3
    n_layers = trial.suggest_int('n_layers', 1, 3)
    model = Sequential()
    # Dodanie pierwszej warstwy ukrytej
    model.add(Dense(trial.suggest_int('n_units_first', 10, 300), activation = 'relu', input_shape = (train_features.shape[1],)))
    # Dodanie warstwy Dropout
    model.add(Dropout(trial.suggest_float('dropout_first', 0.0, 0.5)))

    # Dodawanie kolejnych warstw ukrytych w pętli zależnie od liczby warstw
    for i in range(n_layers):
        model.add(Dense(trial.suggest_int(f'n_units_{i}', 10, 300), activation = 'relu'))
        model.add(Dropout(trial.suggest_float(f'dropout_{i}', 0.0, 0.5)))

    # Dodanie warstwy wyjściowej
    model.add(Dense(2, activation = 'softmax'))
    # Sugestia Optuna dotycząca współczynnika uczenia dla optymalizatora Adama
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    model.compile(optimizer = Adam(learning_rate = lr), loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

# Definicja funkcji celu dla procesu dostrajania Optuna
def objective(trial):
    # Tworzenie modelu z bieżącymi parametrami sugerowanymi przez Optuna
    model = create_model(trial)
    # Ustawienie mechanizmu wczesnego zatrzymywania treningu w celu uniknięcia przeuczenia
    early_stopping = EarlyStopping(monitor = 'val_loss', patience = 10, verbose = 1, restore_best_weights = True)
    model.fit(train_features, train_target, epochs = 50, batch_size = 32, validation_split = 0.2, 
              callbacks = [early_stopping], verbose = 0)
    _, accuracy = model.evaluate(test_features, test_target, verbose = 0)
    return accuracy

# Uruchomienie procesu dostrajania
study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 10)

# Wyświetlenie najlepszych parametrów znalezionych przez Optuna
print('Najlepsze parametry:', study.best_trial.params)

[I 2024-06-06 11:39:20,627] A new study created in memory with name: no-name-a157f879-9aa4-4477-a197-b503544d7e0b
C:\Users\48798\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\48798\AppData\Local\Temp\ipykernel_12852\3357861014.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)


Epoch 16: early stopping
Restoring model weights from the end of the best epoch: 6.


[I 2024-06-06 11:39:26,716] Trial 0 finished with value: 0.5642856955528259 and parameters: {'n_layers': 2, 'n_units_first': 65, 'dropout_first': 0.15017742990782984, 'n_units_0': 273, 'dropout_0': 0.4853181076610907, 'n_units_1': 159, 'dropout_1': 0.0210376745838744, 'lr': 4.298872230855751e-05}. Best is trial 0 with value: 0.5642856955528259.


Epoch 35: early stopping
Restoring model weights from the end of the best epoch: 25.


[I 2024-06-06 11:39:37,120] Trial 1 finished with value: 0.4214285612106323 and parameters: {'n_layers': 3, 'n_units_first': 293, 'dropout_first': 0.09780780586806154, 'n_units_0': 35, 'dropout_0': 0.12979165481128258, 'n_units_1': 174, 'dropout_1': 0.10254261655494107, 'n_units_2': 169, 'dropout_2': 0.369882219175659, 'lr': 0.0009151530986408943}. Best is trial 0 with value: 0.5642856955528259.


...

Epoch 11: early stopping
Restoring model weights from the end of the best epoch: 1.


[I 2024-06-06 11:40:43,737] Trial 8 finished with value: 0.4214285612106323 and parameters: {'n_layers': 2, 'n_units_first': 56, 'dropout_first': 0.2872246723979933, 'n_units_0': 255, 'dropout_0': 0.09516400431096367, 'n_units_1': 115, 'dropout_1': 0.23206946227837205, 'lr': 0.04377974075253523}. Best is trial 6 with value: 0.5785714387893677.


Restoring model weights from the end of the best epoch: 50.


[I 2024-06-06 11:40:55,574] Trial 9 finished with value: 0.4642857015132904 and parameters: {'n_layers': 2, 'n_units_first': 200, 'dropout_first': 0.33208062022810814, 'n_units_0': 186, 'dropout_0': 0.15497705458309669, 'n_units_1': 131, 'dropout_1': 0.28826097609354867, 'lr': 0.0013864271659201476}. Best is trial 6 with value: 0.5785714387893677.


Najlepsze parametry: {'n_layers': 2, 'n_units_first': 205, 'dropout_first': 0.15683790552674343, 'n_units_0': 85, 'dropout_0': 0.20368633884296028, 'n_units_1': 213, 'dropout_1': 0.1685375521867219, 'lr': 0.0001338444154247464}


In [12]:
best_params = {'n_layers': 3, 
               'n_units_first': 105, 
               'dropout_first': 0.1498309254789278, 
               'n_units_0': 78, 
               'dropout_0': 0.33462120080999536, 
               'n_units_1': 147, 
               'dropout_1': 0.43624463704269206, 
               'n_units_2': 126, 
               'dropout_2': 0.42630200041120037, 
               'lr': 0.00041889174933029466}

# Budujemy model sieci neuronowej
model = Sequential()
model.add(Dense(best_params['n_units_first'], activation = 'relu', input_shape = (train_features.shape[1],)))
model.add(Dropout(best_params['dropout_first']))

model.add(Dense(best_params['n_units_0'], activation = 'relu'))
model.add(Dropout(best_params['dropout_0']))

model.add(Dense(best_params['n_units_1'], activation = 'relu'))
model.add(Dropout(best_params['dropout_1']))

model.add(Dense(best_params['n_units_2'], activation = 'relu'))
model.add(Dropout(best_params['dropout_2']))

model.add(Dense(2, activation = 'softmax'))

# Optymalizator Adam
optimizer = Adam(learning_rate = best_params['lr'])

model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

# Dodajemy funkcję, która przerwie trenowanie modelu, gdy model przestaje poprawiać swoją wydajność na zbiorze walidacyjnym
early_stopping = EarlyStopping(monitor = 'val_loss',  
                               patience = 10,         # liczba epok bez poprawy po której trening zostanie zatrzymany
                               verbose = 1,           
                               restore_best_weights = True) 


# Trenowanie modelu z dodaniem callbacku
history = model.fit(train_features, train_target, 
                    epochs = 100, 
                    batch_size = 32, 
                    validation_split = 0.2,
                    callbacks = [early_stopping]) 

# Ocena modelu na danych testowych
test_performance = model.evaluate(test_features, test_target)
print('Test loss:', test_performance[0])
print('Test accuracy:', test_performance[1])

# Ocena modelu na danych treningowych
train_performance = model.evaluate(train_features, train_target)
print('Train loss:', train_performance[0])
print('Train accuracy:', train_performance[1])

Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.5092 - loss: 883119744.0000 - val_accuracy: 0.4286 - val_loss: 231609936.0000
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4867 - loss: 626515648.0000 - val_accuracy: 0.5268 - val_loss: 208329808.0000
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5144 - loss: 425164960.0000 - val_accuracy: 0.4732 - val_loss: 227130160.0000
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5053 - loss: 340051776.0000 - val_accuracy: 0.5625 - val_loss: 253730848.0000
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.5417 - loss: 238019632.0000 - val_accuracy: 0.4464 - val_loss: 151300160.0000
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4997 - loss: 241980000.0000 - val_accuracy: 0.4554 - val_loss: 136961248.0000
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.5192 - loss: 184129808.0000 - val_accuracy: 0.5536 - val_loss: 13202796

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4293 - loss: 7685654.5000 - val_accuracy: 0.4107 - val_loss: 253328.1406
Epoch 48/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5180 - loss: 5715751.5000 - val_accuracy: 0.4018 - val_loss: 297220.4062
Epoch 49/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4875 - loss: 6414539.0000 - val_accuracy: 0.4018 - val_loss: 351258.5938
Epoch 50/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4915 - loss: 5168883.0000 - val_accuracy: 0.4286 - val_loss: 319880.0625
Epoch 51/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5099 - loss: 4762104.0000 - val_accuracy: 0.4107 - val_loss: 265559.2500
Epoch 52/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4727 - loss: 4508340.5000 - val_accuracy: 0.4196 - val_loss: 173501.2188
Epoch 53/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5372 - loss: 4480266.5000 - val_accuracy: 0.4286 - val_loss: 150355.0312
Epoch 54/100
14/14 ━━━━━━━━━━━━━━━━━━━━

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5426 - loss: 1129692.3750 - val_accuracy: 0.5804 - val_loss: 0.6889
Epoch 96/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5363 - loss: 1645109.1250 - val_accuracy: 0.5804 - val_loss: 0.6889
Epoch 97/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5415 - loss: 795677.9375 - val_accuracy: 0.5804 - val_loss: 0.6887
Epoch 98/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5156 - loss: 1379123.5000 - val_accuracy: 0.5804 - val_loss: 0.6887
Epoch 99/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5305 - loss: 3365675.0000 - val_accuracy: 0.5804 - val_loss: 0.6886
Epoch 100/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5405 - loss: 1115771.6250 - val_accuracy: 0.5804 - val_loss: 0.6885
Restoring model weights from the end of the best epoch: 100.
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4156 - loss: 0.6991 
Test loss: 0.6986910104751587
Test accuracy: 0.4214285612106323
18/18 ━━━━━━

In [13]:
# Predykcja na zbiorze treningowym
train_predictions = model.predict(train_features)
train_predictions = np.argmax(train_predictions, axis = 1)
train_true = np.argmax(train_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(train_true, train_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
303
0
256
0
Accuracy (ACC): 0.5420393559928444
True Positive Rate (TPR): 1.0
False Positive Rate (FPR): 1.0
True Negative Rate (TNR): 0.0
Positive Predictive Value (PPV): 0.5420393559928444
Negative Predictive Value (NPV): nan


C:\Users\48798\AppData\Local\Temp\ipykernel_12852\1020962155.py:24: RuntimeWarning: invalid value encountered in scalar divide
  NPV = TN / (TN + FN)


In [14]:
# Predykcja na zbiorze testowym
test_predictions = model.predict(test_features)
test_predictions = np.argmax(test_predictions, axis = 1)
test_true = np.argmax(test_target, axis = 1)

# Obliczanie macierzy pomyłek
cm = confusion_matrix(test_true, test_predictions)
TP = cm[1, 1]  # True Positives
TN = cm[0, 0]  # True Negatives
FP = cm[0, 1]  # False Positives
FN = cm[1, 0]  # False Negatives

print(TP)
print(TN)
print(FP)
print(FN)

# TPR, FPR, TNR, PPV, NPV
ACC = (TP + TN) / (TP + TN + FP + FN)
TPR = TP / (TP + FN)
FPR = FP / (FP + TN)
TNR = TN / (TN + FP)
PPV = TP / (TP + FP)
NPV = TN / (TN + FN)

print('Accuracy (ACC):', ACC)
print('True Positive Rate (TPR):', TPR)
print('False Positive Rate (FPR):', FPR)
print('True Negative Rate (TNR):', TNR)
print('Positive Predictive Value (PPV):', PPV)
print('Negative Predictive Value (NPV):', NPV)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
59
0
81
0
Accuracy (ACC): 0.42142857142857143
True Positive Rate (TPR): 1.0
False Positive Rate (FPR): 1.0
True Negative Rate (TNR): 0.0
Positive Predictive Value (PPV): 0.42142857142857143
Negative Predictive Value (NPV): nan


C:\Users\48798\AppData\Local\Temp\ipykernel_12852\4228670605.py:24: RuntimeWarning: invalid value encountered in scalar divide
  NPV = TN / (TN + FN)
